In [7]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

#
# Prepare the data
#

train = pd.read_csv('train.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader','atm_id']
train.drop(columns, inplace=True, axis=1)

# get the labels
y = train.target.values
train.drop(['target'], inplace=True, axis=1)

x = normalize(train.values)
#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
#
# Create the LightGBM data containers
#

train_data = lightgbm.Dataset(x, label=y)
test_data = lightgbm.Dataset(x_test, label=y_test)

param_grid = {
        'num_leaves': [15, 20, 25],
        #'reg_alpha': [0.1, 0.5],
        'min_data_in_leaf': [30, 50, 100],
        #'lambda_l1': [0, 1, 1.5],
        #'lambda_l2': [0, 1]
        }

estimator = lightgbm.LGBMClassifier(
         boosting_type='gbdt',  
         objective='binary',  
         learning_rate=0.01, 
         metric='binary_logloss')

gridsearch = GridSearchCV(estimator, param_grid)

model=gridsearch.fit(x, y,
        eval_set=[(x_test, y_test)],
        eval_metric=['binary_logloss'],
        early_stopping_rounds=5)

print(model.best_params_, model.best_score_)

#print('Save model...')
# save model to file
# AUC 0.67 - 70 
# AUC 0.65 - 65
#model.save_model('model.txt')

[1]	valid_0's binary_logloss: 0.692145
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.691147
[3]	valid_0's binary_logloss: 0.690195
[4]	valid_0's binary_logloss: 0.689246
[5]	valid_0's binary_logloss: 0.688331
[6]	valid_0's binary_logloss: 0.687465
[7]	valid_0's binary_logloss: 0.686684
[8]	valid_0's binary_logloss: 0.685813
[9]	valid_0's binary_logloss: 0.685006
[10]	valid_0's binary_logloss: 0.684172
[11]	valid_0's binary_logloss: 0.683461
[12]	valid_0's binary_logloss: 0.682777
[13]	valid_0's binary_logloss: 0.681995
[14]	valid_0's binary_logloss: 0.681269
[15]	valid_0's binary_logloss: 0.680587
[16]	valid_0's binary_logloss: 0.679833
[17]	valid_0's binary_logloss: 0.679243
[18]	valid_0's binary_logloss: 0.67854
[19]	valid_0's binary_logloss: 0.677793
[20]	valid_0's binary_logloss: 0.67714
[21]	valid_0's binary_logloss: 0.67655
[22]	valid_0's binary_logloss: 0.675925
[23]	valid_0's binary_logloss: 0.675317
[24]	valid_0's binary_logloss: 

[1]	valid_0's binary_logloss: 0.692224
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.691326
[3]	valid_0's binary_logloss: 0.690393
[4]	valid_0's binary_logloss: 0.689492
[5]	valid_0's binary_logloss: 0.688626
[6]	valid_0's binary_logloss: 0.687691
[7]	valid_0's binary_logloss: 0.686946
[8]	valid_0's binary_logloss: 0.68606
[9]	valid_0's binary_logloss: 0.685242
[10]	valid_0's binary_logloss: 0.684387
[11]	valid_0's binary_logloss: 0.683583
[12]	valid_0's binary_logloss: 0.682842
[13]	valid_0's binary_logloss: 0.682022
[14]	valid_0's binary_logloss: 0.68124
[15]	valid_0's binary_logloss: 0.68052
[16]	valid_0's binary_logloss: 0.679873
[17]	valid_0's binary_logloss: 0.679151
[18]	valid_0's binary_logloss: 0.678395
[19]	valid_0's binary_logloss: 0.677743
[20]	valid_0's binary_logloss: 0.677154
[21]	valid_0's binary_logloss: 0.676477
[22]	valid_0's binary_logloss: 0.675868
[23]	valid_0's binary_logloss: 0.675331
[24]	valid_0's binary_logloss: 

[5]	valid_0's binary_logloss: 0.688446
[6]	valid_0's binary_logloss: 0.687529
[7]	valid_0's binary_logloss: 0.686806
[8]	valid_0's binary_logloss: 0.686078
[9]	valid_0's binary_logloss: 0.685384
[10]	valid_0's binary_logloss: 0.684606
[11]	valid_0's binary_logloss: 0.683837
[12]	valid_0's binary_logloss: 0.683128
[13]	valid_0's binary_logloss: 0.682476
[14]	valid_0's binary_logloss: 0.681842
[15]	valid_0's binary_logloss: 0.681187
[16]	valid_0's binary_logloss: 0.680537
[17]	valid_0's binary_logloss: 0.679955
[18]	valid_0's binary_logloss: 0.679398
[19]	valid_0's binary_logloss: 0.678766
[20]	valid_0's binary_logloss: 0.67818
[21]	valid_0's binary_logloss: 0.677516
[22]	valid_0's binary_logloss: 0.676867
[23]	valid_0's binary_logloss: 0.676338
[24]	valid_0's binary_logloss: 0.67572
[25]	valid_0's binary_logloss: 0.675255
[26]	valid_0's binary_logloss: 0.674728
[27]	valid_0's binary_logloss: 0.674137
[28]	valid_0's binary_logloss: 0.673574
[29]	valid_0's binary_logloss: 0.673068
[30]	va

[4]	valid_0's binary_logloss: 0.688916
[5]	valid_0's binary_logloss: 0.688014
[6]	valid_0's binary_logloss: 0.68704
[7]	valid_0's binary_logloss: 0.686081
[8]	valid_0's binary_logloss: 0.685295
[9]	valid_0's binary_logloss: 0.684458
[10]	valid_0's binary_logloss: 0.683648
[11]	valid_0's binary_logloss: 0.68286
[12]	valid_0's binary_logloss: 0.682089
[13]	valid_0's binary_logloss: 0.681363
[14]	valid_0's binary_logloss: 0.680594
[15]	valid_0's binary_logloss: 0.679675
[16]	valid_0's binary_logloss: 0.67891
[17]	valid_0's binary_logloss: 0.67828
[18]	valid_0's binary_logloss: 0.677495
[19]	valid_0's binary_logloss: 0.676855
[20]	valid_0's binary_logloss: 0.676087
[21]	valid_0's binary_logloss: 0.675284
[22]	valid_0's binary_logloss: 0.674743
[23]	valid_0's binary_logloss: 0.674123
[24]	valid_0's binary_logloss: 0.673448
[25]	valid_0's binary_logloss: 0.672829
[26]	valid_0's binary_logloss: 0.672234
[27]	valid_0's binary_logloss: 0.671631
[28]	valid_0's binary_logloss: 0.671078
[29]	valid

[4]	valid_0's binary_logloss: 0.689414
[5]	valid_0's binary_logloss: 0.688532
[6]	valid_0's binary_logloss: 0.687755
[7]	valid_0's binary_logloss: 0.686796
[8]	valid_0's binary_logloss: 0.685923
[9]	valid_0's binary_logloss: 0.684993
[10]	valid_0's binary_logloss: 0.684112
[11]	valid_0's binary_logloss: 0.683256
[12]	valid_0's binary_logloss: 0.682499
[13]	valid_0's binary_logloss: 0.6816
[14]	valid_0's binary_logloss: 0.680769
[15]	valid_0's binary_logloss: 0.680081
[16]	valid_0's binary_logloss: 0.679377
[17]	valid_0's binary_logloss: 0.678612
[18]	valid_0's binary_logloss: 0.67789
[19]	valid_0's binary_logloss: 0.677256
[20]	valid_0's binary_logloss: 0.676486
[21]	valid_0's binary_logloss: 0.675893
[22]	valid_0's binary_logloss: 0.675275
[23]	valid_0's binary_logloss: 0.67457
[24]	valid_0's binary_logloss: 0.673885
[25]	valid_0's binary_logloss: 0.673146
[26]	valid_0's binary_logloss: 0.672513
[27]	valid_0's binary_logloss: 0.671979
[28]	valid_0's binary_logloss: 0.671313
[29]	valid

...

[6]	valid_0's binary_logloss: 0.687528
[7]	valid_0's binary_logloss: 0.686734
[8]	valid_0's binary_logloss: 0.685843
[9]	valid_0's binary_logloss: 0.685
[10]	valid_0's binary_logloss: 0.684125
[11]	valid_0's binary_logloss: 0.683274
[12]	valid_0's binary_logloss: 0.682511
[13]	valid_0's binary_logloss: 0.681677
[14]	valid_0's binary_logloss: 0.68098
[15]	valid_0's binary_logloss: 0.680186
[16]	valid_0's binary_logloss: 0.679433
[17]	valid_0's binary_logloss: 0.678725
[18]	valid_0's binary_logloss: 0.678016
[19]	valid_0's binary_logloss: 0.677371
[20]	valid_0's binary_logloss: 0.67676
[21]	valid_0's binary_logloss: 0.676081
[22]	valid_0's binary_logloss: 0.675511
[23]	valid_0's binary_logloss: 0.674937
[24]	valid_0's binary_logloss: 0.674362
[25]	valid_0's binary_logloss: 0.673755
[26]	valid_0's binary_logloss: 0.67316
[27]	valid_0's binary_logloss: 0.672633
[28]	valid_0's binary_logloss: 0.672023
[29]	valid_0's binary_logloss: 0.671521
[30]	valid_0's binary_logloss: 0.670971
[31]	valid

[6]	valid_0's binary_logloss: 0.687544
[7]	valid_0's binary_logloss: 0.686757
[8]	valid_0's binary_logloss: 0.685896
[9]	valid_0's binary_logloss: 0.685012
[10]	valid_0's binary_logloss: 0.684215
[11]	valid_0's binary_logloss: 0.68342
[12]	valid_0's binary_logloss: 0.682584
[13]	valid_0's binary_logloss: 0.681893
[14]	valid_0's binary_logloss: 0.681218
[15]	valid_0's binary_logloss: 0.680455
[16]	valid_0's binary_logloss: 0.67968
[17]	valid_0's binary_logloss: 0.678918
[18]	valid_0's binary_logloss: 0.678204
[19]	valid_0's binary_logloss: 0.677458
[20]	valid_0's binary_logloss: 0.67679
[21]	valid_0's binary_logloss: 0.676111
[22]	valid_0's binary_logloss: 0.675524
[23]	valid_0's binary_logloss: 0.674835
[24]	valid_0's binary_logloss: 0.674269
[25]	valid_0's binary_logloss: 0.673667
[26]	valid_0's binary_logloss: 0.673094
[27]	valid_0's binary_logloss: 0.672362
[28]	valid_0's binary_logloss: 0.671647
[29]	valid_0's binary_logloss: 0.671141
[30]	valid_0's binary_logloss: 0.670496
[31]	va

[5]	valid_0's binary_logloss: 0.688592
[6]	valid_0's binary_logloss: 0.687853
[7]	valid_0's binary_logloss: 0.686903
[8]	valid_0's binary_logloss: 0.686029
[9]	valid_0's binary_logloss: 0.68526
[10]	valid_0's binary_logloss: 0.684514
[11]	valid_0's binary_logloss: 0.683867
[12]	valid_0's binary_logloss: 0.683159
[13]	valid_0's binary_logloss: 0.682395
[14]	valid_0's binary_logloss: 0.681668
[15]	valid_0's binary_logloss: 0.680948
[16]	valid_0's binary_logloss: 0.680254
[17]	valid_0's binary_logloss: 0.679652
[18]	valid_0's binary_logloss: 0.678839
[19]	valid_0's binary_logloss: 0.67814
[20]	valid_0's binary_logloss: 0.677557
[21]	valid_0's binary_logloss: 0.676936
[22]	valid_0's binary_logloss: 0.676335
[23]	valid_0's binary_logloss: 0.675785
[24]	valid_0's binary_logloss: 0.67521
[25]	valid_0's binary_logloss: 0.674695
[26]	valid_0's binary_logloss: 0.674166
[27]	valid_0's binary_logloss: 0.67371
[28]	valid_0's binary_logloss: 0.673214
[29]	valid_0's binary_logloss: 0.672715
[30]	vali

[5]	valid_0's binary_logloss: 0.68826
[6]	valid_0's binary_logloss: 0.687293
[7]	valid_0's binary_logloss: 0.686396
[8]	valid_0's binary_logloss: 0.685473
[9]	valid_0's binary_logloss: 0.684576
[10]	valid_0's binary_logloss: 0.683645
[11]	valid_0's binary_logloss: 0.682977
[12]	valid_0's binary_logloss: 0.682111
[13]	valid_0's binary_logloss: 0.681208
[14]	valid_0's binary_logloss: 0.680402
[15]	valid_0's binary_logloss: 0.67959
[16]	valid_0's binary_logloss: 0.678809
[17]	valid_0's binary_logloss: 0.678076
[18]	valid_0's binary_logloss: 0.677336
[19]	valid_0's binary_logloss: 0.676586
[20]	valid_0's binary_logloss: 0.675924
[21]	valid_0's binary_logloss: 0.675264
[22]	valid_0's binary_logloss: 0.674747
[23]	valid_0's binary_logloss: 0.674125
[24]	valid_0's binary_logloss: 0.673641
[25]	valid_0's binary_logloss: 0.673031
[26]	valid_0's binary_logloss: 0.672598
[27]	valid_0's binary_logloss: 0.672025
[28]	valid_0's binary_logloss: 0.671582
[29]	valid_0's binary_logloss: 0.670931
[30]	va

[5]	valid_0's binary_logloss: 0.688264
[6]	valid_0's binary_logloss: 0.687288
[7]	valid_0's binary_logloss: 0.686405
[8]	valid_0's binary_logloss: 0.685469
[9]	valid_0's binary_logloss: 0.684511
[10]	valid_0's binary_logloss: 0.683669
[11]	valid_0's binary_logloss: 0.682788
[12]	valid_0's binary_logloss: 0.68197
[13]	valid_0's binary_logloss: 0.681219
[14]	valid_0's binary_logloss: 0.680411
[15]	valid_0's binary_logloss: 0.679571
[16]	valid_0's binary_logloss: 0.678781
[17]	valid_0's binary_logloss: 0.678064
[18]	valid_0's binary_logloss: 0.67724
[19]	valid_0's binary_logloss: 0.676451
[20]	valid_0's binary_logloss: 0.675819
[21]	valid_0's binary_logloss: 0.675049
[22]	valid_0's binary_logloss: 0.674424
[23]	valid_0's binary_logloss: 0.673839
[24]	valid_0's binary_logloss: 0.673198
[25]	valid_0's binary_logloss: 0.672574
[26]	valid_0's binary_logloss: 0.671967
[27]	valid_0's binary_logloss: 0.671357
[28]	valid_0's binary_logloss: 0.670731
[29]	valid_0's binary_logloss: 0.670087
[30]	va

In [8]:
#
# Create a submission
#
test = pd.read_csv('test.csv',sep=';')
ids = test['atm_id'].values
columns = ['cash_in_out','display_type','scanner_code_reader','atm_id']
test.drop(columns, inplace=True, axis=1)
x = test.values
#submission = pd.read_csv('out.csv')

#submission.drop('ATM_ID', inplace=True, axis=1)

y = model.predict(x)
y = [1 if i >=np.mean(y) else 0 for i in y]
output = pd.DataFrame({'ATM_ID': ids, 'PREDICT': y})
output.to_csv("out.csv", index=False)